In [1]:
from tensorflow import keras
from tensorflow.keras import layers, models, utils, applications, preprocessing

In [2]:
# 導入InceptionV2-有BatchNormalization的Convolution
def con2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1), normalizer=True, activation='relu', name=None):
    """
    Arguments:
        x: input tensor.
        filters: filters in `Conv2D`.
        num_row: height of the convolution kernel.
        num_col: width of the convolution kernel.
        padding: padding mode in `Conv2D`.
        strides: strides in `Conv2D`.
        name: name of the ops; will become name + '_conv' for the convolution, 
              name + '_bn' for the batch norm layer and 
              name + '_act' for the activation layer.
    Returns:
        Output tensor after applying `Conv2D` and `BatchNormalization`
    """
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_bn'
        act_name = name + '_act'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    
    if keras.backend.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    
    x = layers.Conv2D(filters=filters, kernel_size=(num_row, num_col), 
                      strides=strides, padding=padding, use_bias=False, 
                      name=conv_name)(x)
    if normalizer:
        x = layers.BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation:
        x = layers.Activation(activation, name=act_name)(x)
        
    return x

In [6]:
# InceptionV1_block
# Convenience function for 'standard' Inception concatenated blocks
# 產生inception layer 
# specs = ((64,), (96,128), (16,32), (32,)) -> inception-v1 3a層
def concatenated_block(x, specs, channel_axis, name):
    (br0, br1, br2, br3) = specs
    
    branch_0 = con2d_bn(x, br0[0], 1, 1, name=name+"_Branch_0_a_1x1")
    
    branch_1 = con2d_bn(x, br1[0], 1, 1, name=name+"_Branch_1_a_1x1")
    branch_1 = con2d_bn(branch_1, br1[1], 3, 3, name=name+"_Branch_1_b_3x3")
    
    branch_2 = con2d_bn(x, br2[0], 1, 1, name=name+"_Branch_2_a_1x1")
    branch_2 = con2d_bn(branch_2, br2[1], 5, 5, name=name+"_Branch_2_b_5x5")
    
    branch_3 = layers.MaxPooling2D( (3, 3), strides=(1, 1), padding='same', name=name+"_Branch_3_a_max")(x)  
    branch_3 = con2d_bn(branch_3, br3[0], 1, 1, name=name+"_Branch_3_b_1x1")
    
    x = layers.concatenate( [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name=name+"_Concatenated"
    )
    
    return x

In [7]:
# 測試
img_input = layers.Input(shape=(224, 224, 1))
x = concatenated_block(img_input, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1')
x

<tf.Tensor 'Block_1_Concatenated/Identity:0' shape=(None, 224, 224, 256) dtype=float32>

In [8]:
# 將 concatenated_block中nn卷積改為1*n+n*1(導入InceptionV3概念)
def InceptionV3_block(x, specs,channel_axis, name):
    (br0, br1, br2, br3) = specs
    
    branch_0 = con2d_bn(x, br0[0], 1, 1, name=name+"_Branch_0_a_1x1")
    
    branch_1 = con2d_bn(x, br1[0], 1, 1, name=name+"_Branch_1_a_1x1")
    branch_1 = con2d_bn(branch_1, br1[1], 1, 3, name=name+"_Branch_1_b_1x3")
    branch_1 = con2d_bn(branch_1, br1[1], 3, 1, name=name+"_Branch_1_b_3x1")
    
    branch_2 = con2d_bn(x, br2[0], 1, 1, name=name+"_Branch_2_a_1x1")
    branch_2 = con2d_bn(branch_2, br2[1], 1, 5, name=name+"_Branch_2_b_1x5")
    branch_2 = con2d_bn(branch_2, br2[1], 5, 1, name=name+"_Branch_2_b_5x1")
    
    branch_3 = layers.MaxPooling2D( (3, 3), strides=(1, 1), padding='same', name=name+"_Branch_3_a_max")(x)  
    branch_3 = con2d_bn(branch_3, br3[0], 1, 1, name=name+"_Branch_3_b_1x1")

    x = layers.concatenate( [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name=name+"_Concatenated"
    )
    
    return x

In [9]:
# 測試
img_input = layers.Input(shape=(224, 224, 1))
x = InceptionV3_block(img_input, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1')
x

<tf.Tensor 'Block_1_Concatenated_1/Identity:0' shape=(None, 224, 224, 256) dtype=float32>